In [1]:
import os
sys.path.append('/home/josedvq/furnace')
sys.path.append('/home/josedvq/furnace/slowfast')
import torch
import pickle

from torchinfo import summary

/home/josedvq/.virtualenvs/lared/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from lared_dataset.baselines.model.video import (
    make_slow_pretrained_body,
    VideoSegmentationHead)
# from models import (
#     make_slow_pretrained_body,
#     make_slow_pretrained_classifier,
#     make_slow_pretrained_segmenter,
#     SegmentationHead)
from lared_dataset.constants import models_path

/home/josedvq/.virtualenvs/lared/lib/python3.8/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/home/josedvq/.virtualenvs/lared/lib/python3.8/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [4]:
b = make_slow_pretrained_body(pool=False)
s = VideoSegmentationHead(output_len=45)
# s = SegmentationHead(output_len=45)

loaded pre-trained model
missing keys []
unexpected keys ['model.blocks.5.proj.weight', 'model.blocks.5.proj.bias']


In [5]:
input = [
    torch.rand((8, 3, 8, 256, 256)), # slow
    torch.rand((8, 3, 32, 256, 256)) # fast
]

In [6]:
# summary(pretrained, input_data=input, device='cuda', depth=5)
summary(b, input_size=(16, 3, 8, 256, 256), device='cuda', depth=5)

Layer (type:depth-idx)                                  Output Shape              Param #
PTVResNet                                               [16, 2048, 8, 8, 8]       --
├─Net: 1-1                                              [16, 2048, 8, 8, 8]       --
│    └─ModuleList: 2-1                                  --                        --
│    │    └─ResNetBasicStem: 3-1                        [16, 64, 8, 64, 64]       --
│    │    │    └─Conv3d: 4-1                            [16, 64, 8, 128, 128]     (9,408)
│    │    │    └─BatchNorm3d: 4-2                       [16, 64, 8, 128, 128]     (128)
│    │    │    └─ReLU: 4-3                              [16, 64, 8, 128, 128]     --
│    │    │    └─MaxPool3d: 4-4                         [16, 64, 8, 64, 64]       --
│    │    └─ResStage: 3-2                               [16, 256, 8, 64, 64]      --
│    │    │    └─ModuleList: 4-5                        --                        --
│    │    │    │    └─ResBlock: 5-1                 

In [6]:
summary(s, input_size=(16, 2048, 8, 8, 8), device='cuda', depth=5)

Layer (type:depth-idx)                   Output Shape              Param #
SegmentationHead                         [16, 45]                  --
├─Sequential: 1-1                        [16, 1, 8, 1, 1]          --
│    └─AvgPool3d: 2-1                    [16, 2048, 8, 1, 1]       --
│    └─Conv3d: 2-2                       [16, 1, 8, 1, 1]          6,145
├─Sequential: 1-2                        [16, 1, 45]               --
│    └─Flatten: 2-3                      [16, 1, 8]                --
│    └─Upsample: 2-4                     [16, 1, 45]               --
Total params: 6,145
Trainable params: 6,145
Non-trainable params: 0
Total mult-adds (M): 0.79
Input size (MB): 67.11
Forward/backward pass size (MB): 0.00
Params size (MB): 0.02
Estimated Total Size (MB): 67.13